<em><center>PROBLEM STATEMENT:</em></center>
>Advertiser Raw that has not been cleaned and mapped is defined as Unclassified. The objective is to use the Raw Advertiser, campaign and country information to map the Raw field to its correct Advertiser name, Sector 1 and 2. Look for patterns in the data already mapped to a brand and aim to develop a script that automates the process.


In [1]:
    ###IMPORT###
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#### stats
import scipy.stats as stats
#scikitlearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 

    ###DROP, INSPECT, EDIT###
    
#read file
df = pd.read_excel('./BAIS2018.xlsx', sheet_name='BAIS2018')

#drop irrelevant columns
droplist = ['Date','Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Raw fields', 'Advertiser Sector 1', 
           'Platform']
df.drop(droplist, axis=1, inplace=True)

#basic inspection
print('shape : ', df.shape)
print('='*20) 
print('number of unclassified rows : ', len(df[df['Advertiser'] == 'Unclassified']))
print('='*20)
print('number of classified rows : ', len(df[df['Advertiser'] != 'Unclassified' ]))
print('='*20)
df = df[df['Advertiser'] != 'Unclassified' ]

#make all text lowercase
dfcolnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser',
       'Advertiser Sector 2']
df = df[dfcolnames].apply(lambda x: x.str.lower())

shape :  (6338, 5)
number of unclassified rows :  316
number of classified rows :  6022


## PART I - USING ONLY ONE COLUMN TO PREDICT

#### Advertiser

In [2]:
    ###SET UP###

#set up
ad = df[['Advertiser Raw', 'Advertiser']].copy()
adcolnames = ['Advertiser Raw', 'Advertiser']

#remove punctuation
import string
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
ad['Advertiser Raw'] = ad['Advertiser Raw'].apply(lambda x : x.translate(translator))

#filter categories that have less than 1 data point
filtered = ad.groupby('Advertiser')['Advertiser'].filter(lambda x: len(x) > 1)
ad = ad[ad['Advertiser'].isin(filtered)]

    ###TRAIN TEST###
    
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

X = ad['Advertiser Raw']
y = ad['Advertiser']

#initialize model
forest = RandomForestClassifier()
logit = LogisticRegression()
cv = CountVectorizer()
X_1 = cv.fit_transform(X)

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1.toarray(), y, random_state=1, test_size=0.2, stratify=y)

#forest
forest.fit(X_train, y_train)
print('forest score is : ', forest.score(X_test, y_test))

#logistic regression
logit.fit(X_train, y_train)
print('logit score is : ', logit.score(X_test, y_test))

forest score is :  0.9958402662229617
logit score is :  0.9916805324459235


In [3]:
checkad = pd.DataFrame(data=[forest.predict(X_test), y_test], index=['predict', 'actual']).T
checkad[checkad['predict'] != checkad['actual']]

,predict,actual
332,mini,bmw
337,porsche,bentley
637,acura,land rover
809,mini,bmw
931,acura,land rover


#### Advertiser Sector 2

In [4]:
ad2 = df[['Advertiser Raw', 'Advertiser Sector 2']].copy()
ad2colnames = ['Advertiser Raw', 'Advertiser Sector 2']
ad2 = ad2[ad2colnames].apply(lambda x: x.str.lower())

import string
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
ad2['Advertiser Raw'] = ad2['Advertiser Raw'].apply(lambda x : x.translate(translator))

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

X = ad2['Advertiser Raw']
y = ad2['Advertiser Sector 2']

#initialize model
forest = RandomForestClassifier()
logit = LogisticRegression()
cv = CountVectorizer()
X_1 = cv.fit_transform(X)

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1.toarray(), y, random_state=1, test_size=0.2, stratify=y)

#forest
forest.fit(X_train, y_train)
print('forest score is : ', forest.score(X_test, y_test))

#logistic regression
logit.fit(X_train, y_train)
print('logit score is : ', logit.score(X_test, y_test))

forest score is :  0.995850622406639
logit score is :  0.9867219917012449


#### checking on the wrong predictors

In [5]:
checkad2 = pd.DataFrame(data=[forest.predict(X_test), y_test], index=['predict', 'actual']).T
checkad2[checkad2['predict'] != checkad2['actual']]

,predict,actual
488,commercial vehicles,auto parts
498,commercial vehicles,automotive general
587,commercial vehicles,auto parts
911,auto dealerships,commercial vehicles
972,commercial vehicles,auto parts


## PART II - USING MULTIPLE COLUMNS TO PREDICT

#### Set up

In [6]:
###IMPORT###
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#stats
import scipy.stats as stats
#scikitlearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 

    ###DATA CLEANING###

df = pd.read_excel('./BAIS2018.xlsx', sheet_name='BAIS2018')
dfcolnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser',
       'Advertiser Sector 2']
df = df[df['Advertiser'] != 'Unclassified' ]
df = df[dfcolnames].apply(lambda x: x.str.lower())

import string
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
df = df[dfcolnames].applymap(lambda x : x.translate(translator)) #https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_dataframes/

#### Advertiser 

In [7]:
    ###SET UP###

#include only categories that have more than a value count of one
adcolnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser']
filtered = ad.groupby('Advertiser')['Advertiser'].filter(lambda x: len(x) > 1)
ad = df[adcolnames]
ad = ad[ad['Advertiser'].isin(filtered)]

#predictor and target column names
predictors = ['Target Country', 'Campaign', 'Advertiser Raw']
target = ['Advertiser']

In [8]:
def combine_text_columns(data_frame, to_drop=target):
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # Replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

In [9]:
    ###TRAIN TEST##
    
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

X = combine_text_columns(ad[predictors])
y = ad[target].values.ravel() # interesting why i need to ravel if not I'll get an error msg.

#initialize model
forest = RandomForestClassifier()
logit = LogisticRegression()
cv = CountVectorizer()
X_1 = cv.fit_transform(X)

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1.toarray(), y, random_state=1, test_size=0.2, stratify=y)

#forest
forest.fit(X_train, y_train)
print('forest score is : ', forest.score(X_test, y_test))

#logistic regression
logit.fit(X_train, y_train)
print('logit score is : ', logit.score(X_test, y_test))

forest score is :  0.9874686716791979
logit score is :  0.9941520467836257


In [10]:
checkad = pd.DataFrame(data=[forest.predict(X_test), y_test], index=['predict', 'actual']).T
checkad[checkad['predict'] != checkad['actual']]

,predict,actual
0,carsguide,bombardier
38,cadillac,nissan
60,bentley,porsche
100,goget,continental
142,napa auto parts,carmax
145,hyundai,bob jane
491,nissan,seat
617,nissan,mitsubishi
626,subaru,solitaire automotive
1020,goget,hyundai


#### advertiser sector 2

In [11]:
    ###SET UP###
    
ad2colnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser Sector 2']
ad2 = df[ad2colnames]
predictors = ['Target Country', 'Campaign', 'Advertiser Raw']
target = ['Advertiser Sector 2']


    ###TRAIN TEST###
    
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

X = combine_text_columns(ad2[predictors])
y = ad2[target].values.ravel() # interesting why i need to ravel if not I'll get an error msg.

#initialize model
forest = RandomForestClassifier()
logit = LogisticRegression()
cv = CountVectorizer()

X_1 = cv.fit_transform(X)

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_1.toarray(), y, random_state=1, test_size=0.2, stratify=y)

#forest
forest.fit(X_train, y_train)
print('forest score is : ', forest.score(X_test, y_test))

#logistic regression
logit.fit(X_train, y_train)
print('logit score is : ', logit.score(X_test, y_test))

forest score is :  0.9933609958506224
logit score is :  0.995850622406639


In [12]:
checka2 = pd.DataFrame(data=[forest.predict(X_test), y_test], index=['predict', 'actual']).T
checkad2[checkad2['predict'] != checkad2['actual']]

,predict,actual
488,commercial vehicles,auto parts
498,commercial vehicles,automotive general
587,commercial vehicles,auto parts
911,auto dealerships,commercial vehicles
972,commercial vehicles,auto parts


## PART III - PREDICTING THE UNCLASSIFIED

In [13]:
    ###IMPORT###
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#stats
import scipy.stats as stats
#scikitlearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
    
    ###SET UP###
    
#read and basic edit
df = pd.read_excel('./BAIS2018.xlsx', sheet_name='BAIS2018')
droplist = ['Date','Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Raw fields', 'Advertiser Sector 1', 
           'Platform']
df.drop(droplist, axis=1, inplace=True)

#string lower on df
dfcolnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser',
       'Advertiser Sector 2']
df = df[dfcolnames].apply(lambda x: x.str.lower())
import string
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
df = df[dfcolnames].applymap(lambda x : x.translate(translator))

#split df into classified and unclassified
unclassified = df[df['Advertiser'] == 'unclassified'].copy()
classified = df[df['Advertiser'] != 'unclassified' ]

#### model 1: Advertiser

In [14]:
    ###SET UP###
    
#subset dataframe
adcolnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser']
ad = classified[adcolnames]

#predictor and target column names
predictors = ['Target Country', 'Campaign', 'Advertiser Raw']
target = ['Advertiser']

    ###IMPORT###
    
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

    ###CLASSIFIED SECTION###
    
CV = CountVectorizer()
X_train = CV.fit_transform(combine_text_columns(ad[predictors])) #cv.fit_transform ouputs a numpy array
y_train = ad[target].values.ravel() #.ravel to get 1D if not  error msg will result

    ###UNCLASSIFIED SECTION###
    
unclassified_X = unclassified[['Target Country', 'Campaign', 'Advertiser Raw']].copy()
unclassified_X = CV.transform(combine_text_columns(unclassified_X))

logit = LogisticRegression()
logit.fit(X_train, y_train)
predictadvertiser = logit.predict(unclassified_X)
    
    ###ADD TO UNCLASSIFIED DATAFRAME###
    
unclassified['Advertiser'] = predictadvertiser

#### model 2: Advertiser Sector 2

In [15]:
    ###SET UP###
    
#subset dataframe
adsec2_colnames = ['Target Country', 'Campaign', 'Advertiser Raw', 'Advertiser Sector 2']
adsec2 = classified[adsec2_colnames]

#predictor and target column names
predictors = ['Target Country', 'Campaign', 'Advertiser Raw']
target = ['Advertiser Sector 2']

    ###IMPORT###
    
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

    ###CLASSIFIED SECTION###
    
CV = CountVectorizer()
X_train = CV.fit_transform(combine_text_columns(adsec2[predictors])) #cv.fit_transform ouputs a numpy array
y_train = adsec2[target].values.ravel() #.ravel to get 1D if not  error msg will result

    ###UNCLASSIFIED SECTION###
    
unclassified_X = unclassified[['Target Country', 'Campaign', 'Advertiser Raw']].copy()
unclassified_X = CV.transform(combine_text_columns(unclassified_X))

logit = LogisticRegression()
logit.fit(X_train, y_train)
predictadsec2 = logit.predict(unclassified_X)
    
    ###ADD TO UNCLASSIFIED DATAFRAME###
    
unclassified['Advertiser Sector 2'] = predictadsec2
unclassified_predicted = unclassified
unclassified_predicted

,Target Country,Campaign,Advertiser Raw,Advertiser,Advertiser Sector 2
55,united kingdom,nissan aftersales dvla data ret mobile ...,nissan,nissan,commercial vehicles
65,united kingdom,nissan lcv eyeota 10313084,nissan,nissan,commercial vehicles
97,denmark,interdan leasing q1 bt high income,dk mitsubishi,mitsubishi,commercial vehicles
113,germany,eyeota interest auto enthusiasts,citroen de full dfa 2015,citroen,commercial vehicles
115,germany,mercedesbenz pro topdealrange channel news dat...,mercedesbenz de fuel,mercedes benz,luxury vehicles
152,united kingdom,lexus int pro 3rd party data ad ctr mixed rtb ...,lexus pan euro uk zenith international dcm parent,lexus,luxury vehicles
155,united kingdom,mercedes pro 3rd party ad ctr eyeota experian ...,mercedes benz uk zenith ninah,mercedes benz,luxury vehicles
168,united kingdom,ppg omp dev tab bhv exp car age 3 7,bmw,bmw,luxury vehicles
175,united kingdom,toyota uk pro 3rd party ad cpa multi rtb inter...,toyota uk zenith,toyota,commercial vehicles
212,united states,hd 2016 core endemic always on 2016 dsk pro bt...,harley davidson us starcom,harley davidson,motorbikes scooters


In [16]:
unclassified_predicted.to_csv('unclassified_predicted.csv')